# NOTE: THE DIGITAL NUMBERS DON'T WORK, NEED TO CLASSIFY BASED ON SET OF WARZONE-STYLE NUMBERS

In [1]:
import cv2 as cv
import numpy as np 
import pyautogui
from time import time, sleep
# from imutils.perspective import four_point_transform
from imutils import contours
import imutils

DIGITS_LOOKUP = {(1, 1, 1, 0, 1, 1, 1): 0,
                 (0, 0, 1, 0, 0, 1, 0): 1,
                 (1, 0, 1, 1, 1, 1, 0): 2,
                 (1, 0, 1, 1, 0, 1, 1): 3,
                 (0, 1, 1, 1, 0, 1, 0): 4,
                 (1, 1, 0, 1, 0, 1, 1): 5,
                 (1, 1, 0, 1, 1, 1, 1): 6,
                 (1, 0, 1, 0, 0, 1, 0): 7,
                 (1, 1, 1, 1, 1, 1, 1): 8,
                 (1, 1, 1, 1, 0, 1, 1): 9
                }

In [2]:
data_dir = 'media/icons/'
out_dir = 'media/test_kill_count/'

light_needle_img = f'light_kills_counter_skull_icon.jpg'  
light_needle_img_2 = f'light_kills_counter_skull_icon_2.jpg' 
dark_needle_img = f'dark_kills_counter_skull_icon.jpg'
dark_needle_img_2 = f'dark_kills_counter_skull_icon_2.jpg'  

# light_needle_img = cv.imread(light_needle_img, cv.IMREAD_UNCHANGED)
# dark_needle_img = cv.imread(dark_needle_img, cv.IMREAD_UNCHANGED)

n_loops = 120
n_recognized = 0
n_dark_recognized = 0
n_light_recognized = 0
sum_thresh = 0

In [3]:
sleep(10)
now = time()

In [4]:
while(True):
# for _ in range(n_loops):
    loop_time = time()

    # capture screenshot & resize to 720p
    screenshot = pyautogui.screenshot() 
    screenshot = screenshot.resize((1280, 720))
    # translate to opencv
    screenshot = np.array(screenshot)
    screenshot = cv.cvtColor(screenshot, cv.COLOR_RGB2BGR)
    
    # try light then dark background 
    for img in [dark_needle_img, dark_needle_img_2]:  # light_needle_img, light_needle_img_2,  # (never got any correct)
        needle_img = cv.imread(f'{data_dir}{img}', cv.IMREAD_UNCHANGED)
        result = cv.matchTemplate(screenshot, needle_img, cv.TM_CCOEFF_NORMED)

        min_val, max_val, min_loc, max_loc = cv.minMaxLoc(result)

        threshold = 0.82
        if max_val >= threshold:
            print(f'found needle | {str(max_val)[:5]} > {threshold} | {img}')
            n_recognized += 1
            if 'dark' in img:
                n_dark_recognized += 1
            else:
                n_light_recognized += 1
                
            sum_thresh += max_val

            needle_w = needle_img.shape[1]
            needle_h = needle_img.shape[0]

            # tag top left corner, add width & height to find bottom right corner
            top_left = max_loc  # want rectangle
            bottom_right = (top_left[0] + (needle_w * 2), top_left[1] + needle_h)
            
            kills = screenshot[top_left[1]:bottom_right[1], top_left[0]:bottom_right[0]]
            
            '''
            THIS RAISES SERIOUS ISSUES WITH LIGHT BACKGROUNDS 
            -- Need to Make Number more Distinct then change the color is the general vibe
            '''
            # threshold the warped image, then apply a series of morphological
            # operations to cleanup the thresholded image
            gray = cv.cvtColor(kills, cv.COLOR_BGR2GRAY)
            thresh = cv.threshold(gray, 0, 255, cv.THRESH_BINARY_INV | cv.THRESH_OTSU)[1]
            kernel = cv.getStructuringElement(cv.MORPH_ELLIPSE, (1, 5))
            thresh = cv.morphologyEx(thresh, cv.MORPH_OPEN, kernel)
            # cv.imwrite('tst.jpg', gray)
            
            # find contours in the thresholded image, then initialize the
            # digit contours lists
            cnts = cv.findContours(thresh.copy(), cv.RETR_EXTERNAL, cv.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)
            digitCnts = []
            # loop over the digit area candidates
            for c in cnts:
                # compute the bounding box of the contour
                (x, y, w, h) = cv.boundingRect(c)
                # if the contour is sufficiently large, it must be a digit
                if w >= 5 and (h >= 0 and h <= 40):
                    digitCnts.append(c)
                    
            # sort the contours from left-to-right, then initialize the
            # actual digits themselves
            digitCnts = contours.sort_contours(digitCnts, method="left-to-right")[0]
            digits = []
            
            # loop over each of the digits
            for c in digitCnts:
                # extract the digit ROI
                (x, y, w, h) = cv.boundingRect(c)
                roi = thresh[y:y + h, x-(w*5):x + w]
                # compute the width and height of each of the 7 segments
                # we are going to examine
                (roiH, roiW) = roi.shape
                (dW, dH) = (int(roiW * 0.25), int(roiH * 0.15))
                dHC = int(roiH * 0.05)
                # define the set of 7 segments
                segments = [
                    ((0, 0), (w, dH)),  # top
                    ((0, 0), (dW, h // 2)),  # top-left
                    ((w - dW, 0), (w, h // 2)),  # top-right
                    ((0, (h // 2) - dHC) , (w, (h // 2) + dHC)),  # center
                    ((0, h // 2), (dW, h)),  # bottom-left
                    ((w - dW, h // 2), (w, h)),  # bottom-right
                    ((0, h - dH), (w, h))  # bottom
                ]
                on = [0] * len(segments)

                # loop over the segments
                for (i, ((xA, yA), (xB, yB))) in enumerate(segments):
                    # extract the segment ROI, count the total number of
                    # thresholded pixels in the segment, and then compute
                    # the area of the segment
                    segROI = roi[yA:yB, xA:xB]
                    total = cv.countNonZero(segROI)
                    area = (xB - xA) * (yB - yA)
                    # if the total number of non-zero pixels is greater than
                    # 50% of the area, mark the segment as "on"
                    if total / float(area) > 0.5:
                        on[i]= 1
                # lookup the digit and draw it on the image
                digit = DIGITS_LOOKUP[tuple(on)]
                digits.append(digit)
                cv.rectangle(kills, (x, y), (x + w, y + h), (0, 255, 0), 1)
                cv.putText(kills, str(digit), (x + int(0.7*w), y + int(0.5*h)),
                    cv.FONT_HERSHEY_SIMPLEX, 0.65, (0, 255, 0), 2)

#             cv.rectangle(screenshot, top_left, bottom_right, 
#                          color=(0, 255, 0), thickness=2, lineType=cv.LINE_4)

#             # display live
#             cv.imshow('Computer Vision', screenshot) 
            if n_recognized < 10:
                out = f'{out_dir}0{n_recognized}_{img}'
            else:
                out = f'{out_dir}{n_recognized}_{img}'
            print(out)
            print()
#             cv.imwrite(out, screenshot)
            cv.imshow('kills', kills)
            cv.imwrite(out, screenshot)

            print(f'FPS {1 / (time() - loop_time)}')
            loop_time = time()

            if cv.waitKey(1) == ord('q'): 
                cv.destroyAllWindows() 
                break 

        else:
            pass
#             print(f"didn't find needle | {str(max_val)[:5]} < {threshold}")
        
#     sleep(1)

    
print(f'\nDone.\nRecognized: {n_recognized}/{n_loops}\n# Dark recognized: {n_dark_recognized}\n# Light recognized: {n_light_recognized}\nAvg thresh: {sum_thresh / n_recognized}')

found needle | 0.857 > 0.82 | dark_kills_counter_skull_icon_2.jpg
media/test_kill_count/01_dark_kills_counter_skull_icon_2.jpg

FPS 1.970748209822016
found needle | 0.847 > 0.82 | dark_kills_counter_skull_icon_2.jpg
media/test_kill_count/02_dark_kills_counter_skull_icon_2.jpg

FPS 2.0486622140555197
found needle | 0.840 > 0.82 | dark_kills_counter_skull_icon_2.jpg
media/test_kill_count/03_dark_kills_counter_skull_icon_2.jpg

FPS 1.9715448759530323
found needle | 0.831 > 0.82 | dark_kills_counter_skull_icon_2.jpg
media/test_kill_count/04_dark_kills_counter_skull_icon_2.jpg

FPS 2.1372619958766164
found needle | 0.821 > 0.82 | dark_kills_counter_skull_icon_2.jpg
media/test_kill_count/05_dark_kills_counter_skull_icon_2.jpg

FPS 2.0493829332098126
found needle | 0.833 > 0.82 | dark_kills_counter_skull_icon_2.jpg
media/test_kill_count/06_dark_kills_counter_skull_icon_2.jpg

FPS 2.0203036883042818
found needle | 0.833 > 0.82 | dark_kills_counter_skull_icon_2.jpg
media/test_kill_count/07_dark

KeyError: (1, 1, 0, 0, 1, 0, 1)

In [ ]:
then = time()
print(f'runtime: {then - now} seconds')